# Calculating the Implied Earnings Move Using Options Prices

Earnings day can be a pivotal moment for a company's share price. The confluence of expectations and reality is a tradable event, drawing crowds to the options market. Observing the surrounding action can provide insight into the consensus view on, and general sentiment of, the company.

The cost of a straddle - the combined price of an at-the-money call and put - is a common way to gauge the near-term volatility. It's the market's expectation of the price band until expiration. While this includes time value, the isolated price of volatility will generally be higher for the expiry immediately following an earnings release.

Have a look at companies that trade weekly options and are reporting a on Thursday. If they report after the close, the price of the one-day straddle at the bell will be the purest sample of information.

The cells below will demonstrate how to get the data from free sources, using the OpenBB Platform.

In [ ]:
# If using in Google Colab, install the OpenBB library.

#!pip install openbb["all"]

# Restart the runtime before the next block

In [7]:
from datetime import datetime, timedelta

from openbb import obb

obb.user.preferences.output_type = "dataframe"

If the earnings date falls on an option expiry, contracts expiring that day will not provide exposure to the after-market earnings reports.

In [3]:
# Lookup some upcoming earnings dates and sort them by market cap.

earnings_calendar = obb.equity.calendar.earnings(
    start_date=(datetime.now() + timedelta(days=1)).date(),
    end_date=(datetime.now() + timedelta(days=14)).date(),
    provider="nasdaq",
)

earnings_calendar.sort_values(by=["market_cap", "num_estimates"], ascending=False).head(
    20
)

,report_date,symbol,name,eps_previous,eps_consensus,num_estimates,period_ending,previous_report_date,reporting_time,market_cap
166,2024-08-28,NVDA,NVIDIA Corporation,0.25,0.59,13.0,2024-07,2023-08-23,after-hours,3.160887e+12
0,2024-09-05,AVGO,Broadcom Inc.,0.95,0.95,10.0,2024-07,2023-08-31,after-hours,7.716866e+11
167,2024-08-28,CRM,"Salesforce, Inc.",1.63,1.73,16.0,2024-07,2023-08-30,after-hours,2.529962e+11
261,2024-08-26,PDD,PDD Holdings Inc.,1.27,2.66,2.0,2024-06,2023-08-29,pre-market,2.007811e+11
168,2024-08-28,RY,Royal Bank Of Canada,2.13,2.14,3.0,2024-07,2023-08-24,pre-market,1.597315e+11
262,2024-08-26,BHP,BHP Group Limited,NaN,NaN,1.0,2024-06,NaN,after-hours,1.401966e+11
114,2024-08-29,DELL,Dell Technologies Inc.,1.44,1.49,4.0,2024-07,2023-08-31,after-hours,7.922927e+10
169,2024-08-28,CRWD,"CrowdStrike Holdings, Inc.",0.06,0.23,14.0,2024-07,2023-08-30,after-hours,6.648856e+10
225,2024-08-27,BMO,Bank Of Montreal,2.08,1.98,3.0,2024-07,2023-08-29,pre-market,6.319707e+10
115,2024-08-29,MRVL,"Marvell Technology, Inc.",0.18,0.13,13.0,2024-07,2023-08-24,after-hours,6.175190e+10


In [27]:
# Get the options chains data.

symbol = "NVDA"  # This will not be evergreen, change the symbol based on a stock above.

obb.user.preferences.output_type = "OBBject"  # To use the built-in options chains methods, we need to set the output type to OBBject.

options = obb.derivatives.options.chains(symbol, provider="cboe")

last_price = options.results.underlying_price[0]

display(f"Last Price: ${last_price}")

display(options.results.expirations[:3])

'Last Price: $124.7001'

['2024-08-23', '2024-08-30', '2024-09-06']

In [40]:
# Use the straddle method of the results object to get the straddle data and then calculate the implied move.

straddle = options.results.straddle(days=options.results.expirations[1])
straddle_price = straddle.loc["Cost"].values[0]
days = straddle.loc["DTE"].values[0]
upper_price = straddle.loc["Breakeven Upper"].values[0]
lower_price = straddle.loc["Breakeven Lower"].values[0]

implied_move = ((1 + straddle_price / last_price) ** (1 / days) - 1) * 100

display(
    f"Cost of Straddle: ${round(straddle_price, 2)}"
    f"\nCost as a % of Share Price: {round((straddle_price/last_price) * 100, 4)}%"
    f"\nUpper Breakeven Price: ${upper_price}"
    f"\nLower Breakeven Price: ${lower_price}"
    f"\nImplied Daily Move: {round(implied_move, 4)}%\n"
)

display(straddle)

Cost of Straddle: $14.65
Cost as a % of Share Price: 11.7482%
Upper Breakeven Price: $139.65
Lower Breakeven Price: $109.35
Implied Daily Move: 1.3982%



,Long Straddle
Symbol,NVDA
Underlying Price,124.7001
Expiration,2024-08-30
DTE,8
Strike 1,125.0
Strike 2,124.0
Strike 1 Premium,7.55
Strike 2 Premium,7.1
Cost,14.65
Cost Percent,11.7482
